In [3]:
import sys
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
#import collections
import pandas as pd
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# import netCDF4
# from netCDF4 import *

import cartopy.crs as ccrs
import cartopy as cart
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from scipy import interpolate
from scipy.interpolate import griddata
import time
import glob
import dask

from scipy.fftpack import fft
from scipy.fftpack import ifft
import copy
import eofs.standard as Eof_st
from matplotlib.colors import ListedColormap

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

import cmocean
import os

## Create a dummy xarray

In [15]:
datestr=pd.date_range(start='1974-06-02',end='2022-12-31')
x = np.random.rand(len(datestr))
# Create an xarray dataset
MJO_fobs = xr.Dataset({'x': (('time',), x)},
                coords={'time': datestr})

# Print the xarray dataset
MJO_fobs

<xarray.Dataset>
Dimensions:  (time: 17745)
Coordinates:
  * time     (time) datetime64[ns] 1974-06-02 1974-06-03 ... 2022-12-31
Data variables:
    x        (time) float64 0.5694 0.2303 0.02426 ... 0.1947 0.1638 0.1448

In [16]:
base_dir = '/glade/work/wchapman/MJOcast/MJOcast/'
dataNOAA = pd.read_csv('/glade/work/wchapman/MJOcast/MJOcast//Observations/BOM_INDEX.txt', delimiter='\s+', header=0)
dataNOAA.columns = ["year", "month", "day", "RMM1", "RMM2", "phase", 'amplitude', 'doggy']

# Create new variables for the BOM index in the MJO dataset
MJO_fobs['RMM1_obs_BOM'] = copy.deepcopy(MJO_fobs['x'].squeeze())
MJO_fobs['RMM2_obs_BOM'] = copy.deepcopy(MJO_fobs['x'].squeeze())
MJO_fobs['BOM_phase'] = copy.deepcopy(MJO_fobs['x'].squeeze())
MJO_fobs['BOM_amplitude'] = copy.deepcopy(MJO_fobs['x'].squeeze())

# Loop through the time dimension and assign BOM index values to the MJO dataset
for ll in range(len(MJO_fobs['time'])):
    yrnum = int(MJO_fobs['time.year'][ll])
    monum = int(MJO_fobs['time.month'][ll])
    daynum = int(MJO_fobs['time.day'][ll])
    ind_date = dataNOAA[(dataNOAA['year'] == yrnum) & (dataNOAA['month'] == monum) & (dataNOAA['day'] == daynum)].index[0]
    MJO_fobs['RMM1_obs_BOM'][ll:ll+1] = np.array(dataNOAA[ind_date:ind_date+1]['RMM1'])
    MJO_fobs['RMM2_obs_BOM'][ll:ll+1] = np.array(dataNOAA[ind_date:ind_date+1]['RMM2'])
    MJO_fobs['BOM_phase'][ll:ll+1] = np.array(dataNOAA[ind_date:ind_date+1]['phase'])
    MJO_fobs['BOM_amplitude'][ll:ll+1] = np.array(dataNOAA[ind_date:ind_date+1]['amplitude'])

# Save the MJO dataset to a NetCDF fi
MJO_fobs.drop('x').to_netcdf('/glade/scratch/wchapman/data_for_KJM/'+'Observed_MJO_BOMindex.nc')

In [19]:
MJO_fobs.drop('x').to_netcdf('/glade/scratch/wchapman/data_for_KJM/'+'Observed_MJO_BOMindex.nc')